In [112]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import copy

def debugger(key):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
        for a in dataset[key]:
            print(f"{a}")

In [113]:
#headers setting
colnames = ["Tijdstempel","programme","mlcourse","ircourse","statcourse","dbcourse","gender","chatGPT","birthday","studentestimate",
            "stand","stress","sporthours","random","bedtime","goodday1","goodday2"]
dataset = pd.read_csv("./dataset/ODI-2024.csv", skiprows=3, names=colnames)
dataset.drop("Tijdstempel", axis=1, inplace=True)
dataset["mlcourse"] = dataset["mlcourse"].astype("category")
dataset["ircourse"] = dataset["ircourse"].astype("category")
dataset["statcourse"] = dataset["statcourse"].astype("category")
dataset["gender"] = dataset["gender"].astype("category")
dataset["chatGPT"] = dataset["chatGPT"].astype("category")
dataset["stand"] = dataset["stand"].astype("category")
#dataset.head()
original_number=len(dataset)


In [114]:
#program generalize
def generalizeProgramme(x):
    if "bioinformatics" in x or "bisb" in x:
        return "bioinformatics and systems biology"
    if "econometrics" in x:
        return "econometrics and data science"
    if "fintech" in x:
        return "finiancial technology"
    if "computational science" in x:
        return "computational science"
    if "human language technology" in x:
        return "human language technology"
    if "business analytics" in x:
        return "business analytics"
    if "computational finance" in x:
        return "computational finance"
    if "big data" in x:
        if "engineering" in x:
            return "big data engineering"
        else:
            return "big data"
    if "political data journalism" in x:
        return "political data journalism"
    if "quantitative risk management" in x:
        return "quantitative risk management"
    if "software engineering" in x:
        return "software engineering"
    if "cls" in x:
        return "critical language scholarship"
    if " ai " in x or "artificial intelligence" in x:
        if "health" in x:
            return "ai for health"
        else:
            return "artificial intelligence"
    if " cs " in x or "computer science" in x:
        return "computer science"
    if "ba" in x:
        return "bachelor of arts"
    if "mpa" in x:
        return "public administration"
    

dataset["programme"] = dataset["programme"].map(lambda x: x.lower())
dataset["programme"] = dataset["programme"].map(lambda x: f" {x} ")
dataset["programme"] = dataset["programme"].map(generalizeProgramme)
dataset['programme'] = dataset['programme'].fillna('unknown')
dataset["programme"] = dataset["programme"].astype("category")

dataset.head()
print("dropped: ",original_number-len(dataset))


dropped:  0


In [115]:
#stress
tmp = copy.deepcopy(dataset)
def stress_outlier(dataset):
    # Calculate the interquartile range (IQR)
    Q1 = dataset['stress'].quantile(0.25)
    Q3 = dataset['stress'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define the lower and upper bounds for outlier detection
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter out the outliers
    outliers = dataset[(dataset['stress'] < lower_bound) | (dataset['stress'] > upper_bound)]
    
    # Calculate the average stress level excluding the outliers
    non_outlier_data = dataset[(dataset['stress'] >= lower_bound) & (dataset['stress'] <= upper_bound)]
    avg_stress_without_outliers = non_outlier_data['stress'].mean()
    
    # Replace the outliers with the calculated average
    dataset.loc[outliers.index, 'stress'] = avg_stress_without_outliers
    
    return dataset

# Apply the function to replace outliers with the average calculated without outliers
dataset = stress_outlier(dataset)
'''
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    for edited, original in zip(dataset['stress'], tmp['stress']):
            print(f"{original} => {edited}")
'''
dataset.head()
print("dropped: ",original_number-len(dataset))


dropped:  0


In [116]:
#print(dataset.columns)
#sporthours
dataset['sporthours'] = dataset['sporthours'].astype(str).str.extract('(\d+)').astype(float).fillna(0).astype(int)


#student estimate
dataset['studentestimate'] = pd.to_numeric(dataset['studentestimate'], errors='coerce').fillna(0)
dataset['studentestimate'] = dataset['studentestimate'].apply(lambda x: round(x / 20) * 20)
outlier_threshold = 600
# Filter out row with 'studentestimate' values beyond the outlier threshold
dataset = dataset[dataset['studentestimate'] <= outlier_threshold]
dataset['random'] = pd.to_numeric(dataset['random'], errors='coerce').fillna(0)

# ircourses
dataset['ircourse'] = dataset['ircourse'].str.replace('0','no').str.replace('1','yes')


# Estimate
def clean_and_convert_estimate(value):
    # Remove non-numeric characters and spaces
    cleaned_value = re.sub(r'[^\d-]+', '', str(value))
    
    # If the cleaned value contains a hyphen, split it into two numbers
    if '-' in cleaned_value:
        start, end = cleaned_value.split('-')
        try:
            return (int(start) + int(end)) // 2  # Return the average of the range
        except ValueError:
            return None  # Return None if the range cannot be converted
    else:
        # Convert to integer
        try:
            return int(cleaned_value)
        except ValueError:
            return None  # Return None if the value cannot be converted to an integer

dataset['studentestimate'] = dataset['studentestimate'].map(clean_and_convert_estimate)
#dataset.head()
print("dropped: ",original_number-len(dataset))

dropped:  2


In [117]:
#bedtime
tmp= copy.deepcopy(dataset)#debug purpose

def clean_time(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    patterns = [
        r'(\d{1,2}:\d{2})',            # hh:mm format
        r'(\d{1,2}(\.\d{2})?\s*[ap]m)', # hh[.mm] am/pm format
        r'(\d{1,2}\s*(am|pm))',         # hh am/pm format
        r'(\d{1,2})'                    # hh format
    ]
    # Search for time value in the string using each pattern
    for pattern in patterns:
        match = re.search(pattern, x, flags=re.IGNORECASE)
        if match:
            return match.group(1)
    #return np.nan
    return "00:00"

# Function to format time values as hh:mm
def format_time(x):
    if pd.isna(x):
        return np.nan
    parts = x.split(':')
    if len(parts) == 1:  # If only hour is provided
        hour = parts[0].zfill(2)
        return f"{hour}:00"
    elif len(parts) == 2:  # If both hour and minute are provided
        hour = parts[0].zfill(2)
        minute = parts[1].zfill(2)
        return f"{hour}:{minute}"
    else:
        return np.nan


dataset['bedtime'] = dataset['bedtime'].astype(str).str.replace('AM', '').str.replace('PM', '').str.replace('s morgens', '').str.replace('am', '').str.replace('pm', '')
dataset['bedtime'] = dataset['bedtime'].apply(clean_time)
dataset['bedtime'] = dataset['bedtime'].apply(format_time)
print("dropped: ",original_number-len(dataset))

'''
#comparing the pre elaboration with the post to check
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    for edited, original in zip(dataset['bedtime'], tmp['bedtime']):
            print(f"{original} => {edited}")
'''

dropped:  2


'\n#comparing the pre elaboration with the post to check\nwith pd.option_context(\'display.max_rows\', None, \'display.max_columns\', None): \n    for edited, original in zip(dataset[\'bedtime\'], tmp[\'bedtime\']):\n            print(f"{original} => {edited}")\n'

In [120]:
#birthday
def year(x):#losing 2 real years rows
    patterns = [
        r'((\d{4,8}))',
        r'\b(\d{4})\b'
    ]
    for pattern in patterns:
        match = re.search(pattern, x, flags=re.IGNORECASE)
        if match:
            val = str(match.group(1))
            if len(val) > 4:
                val = val[len(match.group(1))-4:]
                
            return val if int(val) >= 1980 and int(val) <= 2003 else np.nan
        return np.nan
    
def month(x):#loses 3
    x = str(x)
    patterns = [
        r'((\d{6,8}))',
        r'(\d{1,4}[-,\/,\.,\ ]\d{1,2})'        
    ]
    # Search for time value in the string using each pattern
    map = {
        "jan": "01",    "feb": "02",    "mar": "03",
        "apr": "04",    "may": "05",    "jun": "06",
        "jul": "07",    "aug": "08",    "sep": "09",
        "oct": "10",    "nov": "11",    "dec": "12"}

    for y in map:
        if y in x.lower():
            return map[y]

    for pattern in patterns:
        match = re.search(pattern, x, flags=re.IGNORECASE)
        if match:
            if(len(str(match.group(1))) !=8):
                val=match.group(1)[len(match.group(1)) -2:].replace('/','').replace('.','').replace('-','')
                val2=match.group(1)[:2].replace('/','').replace('.','').replace('-','')
                final = str(val) if int(val)<=12 and int(val) >=1 else str(val2)
            else:
                val= match.group(1)[2:4]
                val2= match.group(1)[0:2]
                final = str(val) if int(val)<=12 and int(val) >=1 else str(val2)
            #print(f"{final} {final[0]} {final if final[0] != '0' else final[1:]}")
            return final if final[0] != '0' else final[1:]

    return np.nan
    
dataset['byear'] = dataset['birthday'].apply(year)
dataset['bmonth'] = dataset['birthday'].apply(month)
'''
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    for edited, original in zip(dataset['bmonth'], tmp['birthday']):
        if(str(edited) == "nan"):
            print(f"{original} => {edited}")
'''

3 3 3
10 1 10
04 0 4
10 1 10
06 0 6
05 0 5
07 0 7
12 1 12
08 0 8
05 0 5
06 0 6
10 1 10
03 0 3
09 0 9
06 0 6
11 1 11
11 1 11
10 1 10
07 0 7
08 0 8
08 0 8
08 0 8
09 0 9
08 0 8
12 1 12
03 0 3
02 0 2
09 0 9
03 0 3
9 9 9
10 1 10
08 0 8
03 0 3
1 1 1
04 0 4
05 0 5
09 0 9
03 0 3
11 1 11
01 0 1
10 1 10
01 0 1
2 2 2
08 0 8
03 0 3
01 0 1
02 0 2
11 1 11
09 0 9
5 5 5
06 0 6
10 1 10
8 8 8
02 0 2
05 0 5
09 0 9
10 1 10
10 1 10
02 0 2
5 5 5
05 0 5
2 2 2
03 0 3
10 1 10
12 1 12
11 1 11
12 1 12
10 1 10
04 0 4
05 0 5
01 0 1
09 0 9
11 1 11
09 0 9
02 0 2
02 0 2
08 0 8
10 1 10
05 0 5
09 0 9
01 0 1
03 0 3
2 2 2
03 0 3
04 0 4
07 0 7
04 0 4
04 0 4
12 1 12
07 0 7
11 1 11
12 1 12
01 0 1
01 0 1
06 0 6
03 0 3
12 1 12
07 0 7
09 0 9
05 0 5
12 1 12
08 0 8
07 0 7
04 0 4
11 1 11
06 0 6
06 0 6
03 0 3
05 0 5
7 7 7
05 0 5
03 0 3
11 1 11
01 0 1
03 0 3
03 0 3
06 0 6
06 0 6
03 0 3
07 0 7
05 0 5
4 4 4
07 0 7
11 1 11
10 1 10
08 0 8
12 1 12
05 0 5
4 4 4
04 0 4
08 0 8
06 0 6
12 1 12
02 0 2
11 1 11
09 0 9
11 1 11
08 0 8
11 1 11
11 

'\nwith pd.option_context(\'display.max_rows\', None, \'display.max_columns\', None): \n    for edited, original in zip(dataset[\'bmonth\'], tmp[\'birthday\']):\n        if(str(edited) == "nan"):\n            print(f"{original} => {edited}")\n'

In [119]:
dataset.to_csv('./dataset/normalized.csv', index=False)
